In [1]:
import pandas as pd

data = pd.read_csv('games.csv', index_col=0)

In [2]:
# Cleaning / getting data ready for machine learning
data['venue'] = data['venue'].map({'Home' : 1, 'Away' : 0})     # convert venue to 1's and 0's
data['Date'] = pd.to_datetime(data['Date'])
data['opponent'] = data['Opponent'].astype('category').cat.codes # Converting opponent to integers
data = data.drop(columns=['Time'], inplace=False)                # Dropping time column
data = data[data["Team"] != "Arizona Coyotes"]                   # Team doesnt exist anymore

predict = ['venue']

In [3]:
data

,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result,opponent
0,2021-12-12,Anaheim Ducks,St. Louis Blues,0,17010.0,3,2,39,7.7,0.920,2,25,8.0,0.923,2,1,25
1,2023-01-28,Anaheim Ducks,Arizona Coyotes,1,16126.0,2,1,45,4.4,0.971,20,34,2.9,0.956,10,1,1
2,2022-01-14,Anaheim Ducks,Minnesota Wild,0,18300.0,3,7,42,7.1,0.793,5,42,16.7,0.929,7,0,14
3,2024-11-05,Anaheim Ducks,Vancouver Canucks,1,13538.0,1,5,22,4.5,0.865,10,37,13.5,0.955,8,0,29
4,2022-10-12,Anaheim Ducks,Seattle Kraken,1,17530.0,5,4,27,18.5,0.917,15,48,8.3,0.815,11,1,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,2024-01-02,Winnipeg Jets,Tampa Bay Lightning,1,14157.0,4,2,28,14.3,0.941,6,34,5.9,0.889,6,1,26
10492,2025-03-07,Winnipeg Jets,New Jersey Devils,0,16088.0,6,1,35,17.1,0.957,4,23,4.3,0.829,4,1,17
10493,2023-02-20,Winnipeg Jets,New York Rangers,0,18006.0,4,1,21,19.0,0.980,13,51,2.0,0.810,7,1,19
10494,2025-03-09,Winnipeg Jets,Carolina Hurricanes,0,18700.0,2,4,22,9.1,0.885,24,27,14.8,0.909,8,0,5


In [4]:
# Function to compute rolling averages over the last 3 games. Games for which there isnt enough data (i.e. the first 3 games of each teams) are dropped

def rolling_averages(team, cols, new_cols, window=3):
    team = team.sort_values("Date")    # Getting team data organized chronologically
    rolling = team[cols].rolling(window, closed='left').mean()   # closed=left to ignore current row in sliding window
    team[new_cols] = rolling
    team = team.dropna(subset=new_cols) # dropping first rows because not enough data
    return team

In [5]:
cols = ['G', 'GA', 'S', 'SV%', 'S%']   # wanted columns for rolling
new_cols = [f"{c}_rolling" for c in cols]
predictors = new_cols + predict                         # New predictors


games_data = data.groupby('Team').apply(lambda x: rolling_averages(x, cols, new_cols, 3))   # Compute rolling averages
games_data = games_data.droplevel("Team")
games_data.index = range(games_data.shape[0])  # fixing index level
games_data

C:\Users\vikto\AppData\Local\Temp\ipykernel_29976\322024329.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  games_data = data.groupby('Team').apply(lambda x: rolling_averages(x, cols, new_cols, 3))   # Compute rolling averages


,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,...,SA%,Opponent SV%,Opponent PIM,Result,opponent,G_rolling,GA_rolling,S_rolling,SV%_rolling,S%_rolling
0,2021-10-19,Anaheim Ducks,Edmonton Oilers,0,14082.0,5,6,36,13.9,0.861,...,16.2,0.733,4,0,11,2.666667,1.666667,26.000000,0.959000,10.900000
1,2021-10-21,Anaheim Ducks,Winnipeg Jets,0,13886.0,1,5,39,2.6,0.846,...,18.5,0.974,6,0,32,3.000000,3.333333,30.666667,0.922333,9.466667
2,2021-10-23,Anaheim Ducks,Minnesota Wild,0,18055.0,3,4,24,12.5,0.889,...,11.1,0.875,10,0,14,3.000000,4.333333,34.000000,0.886667,9.200000
3,2021-10-26,Anaheim Ducks,Winnipeg Jets,1,11951.0,3,4,35,8.6,0.840,...,16.0,0.914,6,0,32,3.000000,5.000000,33.000000,0.865333,9.666667
4,2021-10-28,Anaheim Ducks,Buffalo Sabres,1,12014.0,3,4,37,8.1,0.862,...,13.8,0.919,2,0,3,2.333333,4.333333,32.666667,0.858333,7.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10146,2025-04-07,Winnipeg Jets,St. Louis Blues,1,15225.0,3,1,26,11.5,0.933,...,6.7,0.920,2,1,25,2.000000,2.666667,24.666667,0.910333,8.833333
10147,2025-04-10,Winnipeg Jets,Dallas Stars,0,18532.0,4,0,35,11.4,1.000,...,0.0,0.886,7,1,9,2.666667,1.666667,27.000000,0.940667,10.900000
10148,2025-04-12,Winnipeg Jets,Chicago Blackhawks,0,20634.0,5,4,42,9.5,0.875,...,12.5,0.905,4,1,6,2.666667,1.666667,31.333333,0.940667,8.633333
10149,2025-04-13,Winnipeg Jets,Edmonton Oilers,1,15225.0,1,4,18,5.6,0.921,...,10.3,0.944,4,0,11,4.000000,1.666667,34.333333,0.936000,10.800000


In [ ]:
games_data = games_data[
        (games_data['Team'] != 'Arizona Coyotes') &
        (games_data['Opponent'] != 'Arizona Coyotes')
        ]

In [ ]:
#####
# 1. Initialize Elo ratings
initial_elo = 1500
teams = games_data['Team'].unique()
elo_ratings = {team: initial_elo for team in teams}

elo_features = []

# 2. Loop through each game and update ratings
for idx, row in games_data.iterrows():
    team = row['Team']
    opponent = row['Opponent']
    venue = row['venue']
    result = row['Result']  # 1 if win, 0 if loss

    # Optional: home-ice advantage
    team_elo = elo_ratings[team] + (35 if venue == 'Home' else 0)
    opponent_elo = elo_ratings[opponent]

    # Store Elo features BEFORE the game
    elo_features.append({
        'team_elo': team_elo,
        'opponent_elo': opponent_elo,
        'elo_diff': team_elo - opponent_elo
    })

    # Calculate expected outcome
    expected_win = 1 / (1 + 10 ** ((opponent_elo - team_elo) / 400))

    # Elo update (K-factor can be tuned)
    k = 30
    change = k * (result - expected_win)
    elo_ratings[team] += change
    elo_ratings[opponent] -= change

# Convert Elo features to DataFrame
elo_df = pd.DataFrame(elo_features)

# Merge with combined_team_view
games_data = pd.concat([games_data.reset_index(drop=True), elo_df], axis=1)

####
new = ['team_elo', 'opponent_elo', 'elo_diff']
predictors = predictors + new

In [ ]:
games_data

In [ ]:
games_data['days_since_last'] = games_data.groupby('Team')['Date'].diff().dt.days
games_data['days_since_last'].fillna(-1, inplace=True)

games_data['goal_diff'] = games_data['G'] - games_data['GA']

test = ['goal_diff']
new_col = ['goal_diff_rolling']

games_data = games_data.groupby('Team').apply(lambda x: rolling_averages(x, test, new_col, 5))
games_data = games_data.droplevel('Team')
games_data.index = range(len(games_data))

games_data

In [ ]:
new_pred = ['days_since_last']
predictors = predictors + new_pred
predictors

In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression

model = XGBClassifier(random_state=10, )    # base model

# Defining Time Series Split
TSS = TimeSeriesSplit(n_splits=5)

test_model = RandomForestClassifier(random_state=10)

lin = BaggingClassifier(LogisticRegression(random_state=10, solver='liblinear', penalty='l2', max_iter = 1000))


In [ ]:
from sklearn.metrics import precision_score

# Function to make predictions given the data, input features and chosen model

def make_predictions(data, predictors, model):
    train = data[data['Date'] < '2024-04-19']
    #train = train[train['Date'] > '2022-10-06']
    test = data[data['Date'] > '2024-04-19']
    model.fit(train[predictors], train['Result'])
    preds = model.predict(test[predictors])
    combined  = pd.DataFrame(dict(actual=test['Result'], prediction = preds), index=test.index)
    precision = precision_score(test['Result'], preds)
    return combined, precision

In [ ]:
# Defining search space for GridSearchCV
search_grid = {
    'n_estimators': [50, 75, 100, 150],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.02, 0.03],
    'reg_alpha': [1, 5, 10],
    'reg_lambda': [1, 5, 10]

}

alt_search_grid = {
    'n_estimators' : [50, 100, 200],
    'max_depth' : [3, 6, 9],
    'min_samples_split': [3, 5, 10]
}

lin_search_grid = {
    # Logistic Regression hyperparameters (base_estimator__)
    'estimator__C' : [0.5, 0.8, 1.0],
    'n_estimators': [10, 50, 100, 150, 200],
}

GS = GridSearchCV(
    estimator = test_model,
    param_grid = alt_search_grid,
    scoring = 'neg_log_loss',
    refit = True,
    cv = TSS,
    verbose= 4
)

training = games_data[games_data['Date'] < '2024-04-19']  # Training using 2021-2024 data
#training = training[training['Date'] > '2022-10-06']
testing = games_data[games_data['Date'] > '2024-04-19']   # Testing on most recent season (2024-2025)


In [ ]:
GS.fit(training[predictors], training['Result'])     # Training

In [ ]:
GS.best_score_

In [ ]:
new_model = GS.best_estimator_
new_model

In [ ]:
combined, precision = make_predictions(games_data, predictors, new_model)
precision

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, log_loss
predictions = new_model.predict(testing[predictors])

print(classification_report(testing['Result'], predictions))

In [ ]:
# Create DataFrame pairing features with their importances
importances = pd.DataFrame({
    'Feature': predictors,
    'Importance': new_model.feature_importances_
})

# Sort by importance
importances = importances.sort_values(by='Importance', ascending=False)

# Display top features
print(importances.head(10))


In [ ]:
combined

In [ ]:
combined = combined.merge(games_data[['Date', 'Team', 'Opponent', 'Result']], left_index=True, right_index=True)
combined

In [ ]:
final = combined.merge(combined, left_on=['Date', 'Team'], right_on=['Date', 'Opponent'])  # few games will drop due to rolling windows
final

In [ ]:
final[(final['prediction_x'] == 1) & (final['prediction_y'] == 0)]['actual_x'].value_counts()

In [ ]:
final[(final['prediction_x'] == 0) & (final['prediction_y'] == 1)]['actual_y'].value_counts()

In [ ]:
print("Overall accuracy turned out to be ~60%.")
print("However, when merging predictions for both games, we can see that the model is 63.3% accurate when both teams predictions match.")

In [ ]:
total = 560 + 390
total

In [ ]:
560/total